In [1]:
import os
import cv2
import pytesseract
from pytesseract import Output
import json
from fuzzywuzzy import fuzz 
import numpy as np
from sklearn.linear_model import LinearRegression

import asyncio
import requests
from concurrent.futures import ThreadPoolExecutor
import json
import pandas as pd
import os

/home/leo/.local/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def ExtractData(img):
    d = pytesseract.image_to_data(img, output_type=Output.DICT, config="-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz -psm 6")
    return d

In [3]:
masked_words={'VAMOS':21,'TODOS':3,'UNE':6,'UNIONISTA':7,'CONVERGENCIA':13,'CREO':14,'FCNNACION':12,'WINAQ':16,'PPT':19,'UNIDOS':18,'EG':10,'URNGMAIZ':5,'VICTORIA':15,'PHG':23,'VIVA':11,'AVANZA':22,'LIBRE':27,'PANPODEMOS':168,'MLP':24}
#! ordered as in the acta
all_codes=[21,3,6,7,13,14,12,16,19,18,10,5,15,23,11,22,27,168,24]

In [4]:
def MaskWords(img,d):
    party_codes={}
    p1d=(450, 100)
    p2d=(1000, 100)
    cv2.putText(img, "A", (p1d[0], p1d[1]),cv2.FONT_HERSHEY_SIMPLEX, 4, (255, 0, 0), 3)
    cv2.putText(img, "B", (p2d[0], p2d[1]),cv2.FONT_HERSHEY_SIMPLEX, 4, (255, 0, 0), 3)

    n_boxes = len(d['level'])
    for i in range(n_boxes):
            maskw=False
            if len(d['text'][i])>1:
                d['text'][i]=d['text'][i].replace('0','O')
                d['text'][i]=''.join([i for i in d['text'][i] if i.isalpha()])
                d['text'][i]=d['text'][i].upper()
                #if text is directly found in the list
                if d['text'][i] in masked_words:
                    maskw=True
                    code=masked_words[d['text'][i]]
                    if code in party_codes:
                            if(d['left'][i]<party_codes[code][0]):
                                party_codes[code]=(d['left'][i], d['top'][i], d['width'][i], d['height'][i])
                    else:
                        party_codes[code]=(d['left'][i], d['top'][i], d['width'][i], d['height'][i])
                #else: #if not do fuzzy matching
                #    for mw in masked_words:                        
                #        if fuzz.ratio(d['text'][i], mw) > 95:
                #            code=masked_words[mw]
                #            if code in party_codes:
                #                if(d['left'][i]<party_codes[code][0]):
                #                    party_codes[code]=(d['left'][i], d['top'][i], d['width'][i], d['height'][i])
                #            else:
                #                party_codes[code]=(d['left'][i], d['top'][i], d['width'][i], d['height'][i])
                #            #print("Fcodigo ",code,d['text'][i],(d['left'][i], d['top'][i], d['width'][i], d['height'][i]))
                #            maskw=True
                #            break            
            if maskw:
                (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])                
                #cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), -1)
                cv2.rectangle(img, (0, y), (x + w, y + h), (0, 255, 0), -1)
    add_extra=False    
    for code in all_codes:
        if code not in party_codes:
            add_extra=True
    if add_extra and len(party_codes)>1:
        
        xc=[]
        c=[]
        yc=[]
        for k,val in party_codes.items():
            c.append(all_codes.index(k))
            yc.append(val[1])
            xc.append(val[0])
        c=np.array(c).reshape(-1, 1)
        yc=np.array(yc).reshape(-1, 1)
        xc=np.array(xc).reshape(-1, 1)
        regYC = LinearRegression().fit(c,yc)
        regXC = LinearRegression().fit(c,xc)
        for x,code in enumerate(all_codes):           
            if code not in party_codes:
                xa=np.array(x).reshape(-1, 1)
                yc=regYC.predict(xa)
                yc=int(np.asscalar(yc))
                xc=regXC.predict(xa)
                xc=int(np.asscalar(xc))
                xc=0
                
                cv2.rectangle(img, (0, yc), (xc+250,yc+20), (0, 255, 0), -1)
                #cv2.rectangle(img, (xc, yc), (xc+170,yc+20), (0, 255, 0), -1)
                
                
                #cv2.putText(img2, "A"+str(x), (xc, yc+20),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 3)
            #else:
                
            #    xc=party_codes[code][0]
            #    yc=party_codes[code][1]
            #    cv2.putText(img2, "A"+str(x), (xc, yc+20),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 3)
    return img

In [5]:
def DrawBoxes(img,d):
    img2=img.copy()
    n_boxes = len(d['level'])
    for i in range(n_boxes):
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            cv2.rectangle(img2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return img2

In [6]:
def GetBoundingBox(d):
    first_w='PAPELETAS'
    last_w='Municipal'
    last_pad=(200,0)
    first_pad=(0,0)
    p1d=(0, 500)
    p2d=(1300, 2100)
         
         

    
    idx_f=GetIndex(d,first_w,True)
    idx_l=GetIndex(d,last_w,False)

    if idx_f!=-1:
        #p1= (d['left'][idx_f]+first_pad[0], d['top'][idx_f]+first_pad[1])
        p1= (0, d['top'][idx_f]+first_pad[1])
    else:
        
        p1= p1d#default value
        
    if idx_l!=-1:
        p2=(d['left'][idx_l]+d['width'][idx_l]+last_pad[0]//3, d['top'][idx_l]+d['height'][idx_l]+last_pad[1])
    else:
        p2=p2d #default value
    
    if p2[0]-p1[0]<20 or p2[1]-p1[1]<20:
        p2=p2d
        p1= p1d
    
    return (p1,p2)

In [7]:
def GetIndex(d,word,first=True):
    if word in d['text']:
        if first:
            return d['text'].index(word)
        else:
            return len(d['text']) - 1 - d['text'][::-1].index(word)
    else:
        return -1

In [8]:
def ProcessFile(mesa):
    print("Mesa",mesa)
    boleta=1
    fname='{0:06d}'.format(mesa*10+boleta)+'.jpg'
    fname2='test_{0:06d}'.format(mesa*10+boleta)+'.jpg'
    path="data/"
    out_path='./data_mturk_pres/'
    
    p1=(380,720)
    p2=(600,1950)
    
    exists = os.path.isfile(path+fname)
    if not exists:
        print("ACTA ",fname," NO EXISTE")
        return (True,-1)
   
    image = cv2.imread(path+fname) 
    
    #Extract data for bounding box
    d=ExtractData(image)
    
    (p1,p2)=GetBoundingBox(d)
    #print(p1)
    #print(p2)
    
    
    
    #image=DrawBoxes(image,d)

    crop_img = image[p1[1]:p2[1], p1[0]:p2[0]]
    
    #Extract data for masking
    d=ExtractData(crop_img)
    crop_img=MaskWords(crop_img,d)
    
    cv2.imwrite(out_path+fname,crop_img)
    #cv2.imwrite(out_path+fname2,image)
    return

In [9]:
ProcessFile(1)

Mesa 1


In [10]:
#tot_mesas=21100
#bad_mesa=[]
#for mesa in range(1,tot_mesas+1):    
#    ProcessFile(mesa)

In [11]:
async def get_data_asynchronous(path="data/",startFromZero=True):
    
    mesas=[]
    if startFromZero:
        mesas=range(1,total_mesas+1)
    else:
        df=pd.read_csv('data/results.csv')
        for index, row in df.iterrows():
            if row['datos']==False or row['actas']<5:
                mesas.append(int(row['mesa']))
    
    with ThreadPoolExecutor(max_workers=2) as executor:
        loop = asyncio.get_event_loop()
        tasks = [
                loop.run_in_executor(
                    executor,
                    ProcessFile,
                    mesa
                )
            for mesa in mesas
        ]
        res=await asyncio.gather(*tasks)

In [ ]:
total_mesas=21100
loop = asyncio.get_event_loop()
future = asyncio.ensure_future(get_data_asynchronous())
loop.run_until_complete(future)

Mesa 1
Mesa 2
Mesa 3
Mesa 4
Mesa 5
Mesa 6
Mesa 7
Mesa 8
Mesa 9
Mesa 10
Mesa 11
Mesa 12
Mesa 13
Mesa 14
Mesa 15
Mesa 16
Mesa 17
Mesa 18
Mesa 19
Mesa 20
Mesa 21
Mesa 22
Mesa 23
Mesa 24
Mesa 25
Mesa 26
Mesa 27
Mesa 28
Mesa 29
Mesa 30
Mesa 31
Mesa 32
Mesa 33
Mesa 34
Mesa 35
Mesa 36
Mesa 37
Mesa 38
Mesa 39
Mesa 40
Mesa 41
Mesa 42
Mesa 43
Mesa 44
Mesa 45
Mesa 46
Mesa 47
Mesa 48
Mesa 49
Mesa 50
Mesa 51
Mesa 52
Mesa 53
Mesa 54
Mesa 55
Mesa 56
Mesa 57
Mesa 58
Mesa 59
Mesa 60
Mesa 61
Mesa 62
Mesa 63
Mesa 64
Mesa 65
Mesa 66
Mesa 67
Mesa 68
Mesa 69
Mesa 70
Mesa 71
Mesa 72
Mesa 73
Mesa 74
Mesa 75
Mesa 76
Mesa 77
Mesa 78
Mesa 79
Mesa 80
Mesa 81
Mesa 82
Mesa 83
Mesa 84
Mesa 85
Mesa 86
Mesa 87
Mesa 88
Mesa 89
Mesa 90
Mesa 91
Mesa 92
Mesa 93
